# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
# Useful imports
import os
import hydra
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook


/tmp/ipykernel_8331/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

# Simulating the planner <a name="simulation"></a>

In [3]:
%pwd

'/host_home/Repos/nuplan-devkit/experiments'

In [4]:
%env NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps

env: NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps


## Prepare the simulation config

In [5]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(os.getenv('NUPLAN_TUTORIAL_PATH', ''), '../nuplan/planning/script')
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2.5h of 8 autolabeled logs in Las Vegas)
    'scenario_filter=one_continuous_log',  # simulate only one log
    "scenario_filter.log_names=['2021.06.14.16.48.02_veh-12_04057_04438']",
    'scenario_filter.limit_total_scenarios=1',  # use 1 total scenarios
]
ckpt_dir = '/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt'
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
#"/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
#"/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(config_name=simulation_hydra_paths.config_name, overrides=[
    '+simulation=closed_loop_reactive_agents',
    #'model=pgm_hybrid_model',
    'planner=pdm_hybrid_planner',
    f"planner.pdm_hybrid_planner.checkpoint_path={ckpt_dir}" ,
    #'planner.ml_planner.model_config=${model}',
    #f'planner.ml_planner.checkpoint_path={ckpt_dir}',
    #f'observation=idm_agents_observation',
    #'observation.model_config=${model}',
    #f'observation.checkpoint_path={ckpt_dir}',
    'worker=sequential',
    '+occlusion=true',
    '+occlusion.manager_type=complete_shadow', #options: [range, complete_shadow]
    "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
    *DATASET_PARAMS,
])

## Launch simulation (within the notebook)

In [6]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import AGENT_TYPES, STATIC_OBJECT_TYPES, TrackedObjectType

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2023-11-29 23:28:28,178 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-29 23:28:28,179 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2023-11-29 23:28:28,179 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2023-11-29 23:28:28,180 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-11-29 23:28:28,180 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2023-11-29 23:28:28,180 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: /home/ehdykh

In [7]:
runner = runners[0]

In [8]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific callback
    runner._simulation.callback.on_planner_start(runner.simulation.setup, runner.planner)

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(runner.simulation.setup, runner.planner, trajectory)

    iteration = runner.simulation._time_controller.get_iteration()

    runner.simulation.propagate(trajectory)

    # Execute specific callback
    runner.simulation.callback.on_step_end(runner.simulation.setup, runner.planner, runner.simulation.history.last())

runner.simulation.callback.on_simulation_end(runner.simulation.setup, runner.planner, runner.simulation.history)

build_wedge 6.103515625e-05
collect_targets 0.01638007164001465
sort_time 1.0967254638671875e-05
build_wedge 2.7418136596679688e-05
collect_targets 0.01570582389831543
sort_time 7.867813110351562e-06
build_wedge 2.6464462280273438e-05
collect_targets 0.015442848205566406
sort_time 9.298324584960938e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.015385627746582031
sort_time 9.059906005859375e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.0154571533203125
sort_time 9.059906005859375e-06
build_wedge 2.6226043701171875e-05
collect_targets 0.015446662902832031
sort_time 9.059906005859375e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.015524148941040039
sort_time 1.0251998901367188e-05
build_wedge 2.5987625122070312e-05
collect_targets 0.015358686447143555
sort_time 2.5033950805664062e-05
build_wedge 2.7418136596679688e-05
collect_targets 0.01532745361328125
sort_time 1.1682510375976562e-05
build_wedge 2.6226043701171875e-05
collect_targets 0.01546025276184082
s

collect_targets 0.015837907791137695
sort_time 5.9604644775390625e-06
build_wedge 0.0001251697540283203
collect_targets 0.01511383056640625
sort_time 9.775161743164062e-06
build_wedge 2.956390380859375e-05
collect_targets 0.01521611213684082
sort_time 1.0967254638671875e-05
build_wedge 2.956390380859375e-05
collect_targets 0.015129566192626953
sort_time 9.298324584960938e-06
build_wedge 2.9325485229492188e-05
collect_targets 0.015078306198120117
sort_time 1.0013580322265625e-05
build_wedge 4.5299530029296875e-05
collect_targets 0.015068531036376953
sort_time 9.059906005859375e-06
build_wedge 2.9802322387695312e-05
collect_targets 0.01505279541015625
sort_time 9.059906005859375e-06
build_wedge 2.956390380859375e-05
collect_targets 0.015053510665893555
sort_time 1.049041748046875e-05
build_wedge 3.0040740966796875e-05
collect_targets 0.015007257461547852
sort_time 1.0728836059570312e-05
build_wedge 2.9802322387695312e-05
collect_targets 0.01505732536315918
sort_time 9.059906005859375e-06

collect_targets 0.01559591293334961
sort_time 2.0503997802734375e-05
build_wedge 3.123283386230469e-05
collect_targets 0.015277862548828125
sort_time 5.0067901611328125e-06
build_wedge 2.4557113647460938e-05
collect_targets 0.015142440795898438
sort_time 5.0067901611328125e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.015144586563110352
sort_time 6.67572021484375e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.015144109725952148
sort_time 4.76837158203125e-06
build_wedge 2.574920654296875e-05
collect_targets 0.01517939567565918
sort_time 4.5299530029296875e-06
build_wedge 2.5033950805664062e-05
collect_targets 0.015273809432983398
sort_time 4.76837158203125e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.015196561813354492
sort_time 5.0067901611328125e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.015115022659301758
sort_time 5.0067901611328125e-06
build_wedge 2.574920654296875e-05
collect_targets 0.015096187591552734
sort_time 7.152557373046875e-

collect_targets 0.015508413314819336
sort_time 9.059906005859375e-06
build_wedge 0.00012493133544921875
collect_targets 0.01456594467163086
sort_time 1.1682510375976562e-05
build_wedge 2.9802322387695312e-05
collect_targets 0.014702558517456055
sort_time 1.0251998901367188e-05
build_wedge 2.9087066650390625e-05
collect_targets 0.015093803405761719
sort_time 7.867813110351562e-06
build_wedge 2.9802322387695312e-05
collect_targets 0.014778852462768555
sort_time 6.67572021484375e-06
build_wedge 2.86102294921875e-05
collect_targets 0.01464390754699707
sort_time 6.198883056640625e-06
build_wedge 2.8848648071289062e-05
collect_targets 0.014564275741577148
sort_time 6.4373016357421875e-06
build_wedge 2.9087066650390625e-05
collect_targets 0.014760255813598633
sort_time 6.198883056640625e-06
build_wedge 2.8848648071289062e-05
collect_targets 0.014768362045288086
sort_time 5.0067901611328125e-06
build_wedge 2.86102294921875e-05
collect_targets 0.01473855972290039
sort_time 4.76837158203125e-06


collect_targets 0.015591859817504883
sort_time 5.9604644775390625e-06
build_wedge 5.9604644775390625e-05
collect_targets 0.014839410781860352
sort_time 4.5299530029296875e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.014910697937011719
sort_time 4.76837158203125e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.014789104461669922
sort_time 4.5299530029296875e-06
build_wedge 2.574920654296875e-05
collect_targets 0.014714717864990234
sort_time 2.384185791015625e-07
build_wedge 2.4318695068359375e-05
collect_targets 0.014714717864990234
sort_time 4.76837158203125e-07
build_wedge 2.47955322265625e-05
collect_targets 0.014760255813598633
sort_time 7.152557373046875e-07
build_wedge 2.4557113647460938e-05
collect_targets 0.014728069305419922
sort_time 4.76837158203125e-07
build_wedge 2.4080276489257812e-05
collect_targets 0.014676809310913086
sort_time 5.245208740234375e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.014773130416870117
sort_time 5.0067901611328125e-

...

collect_targets 0.014897823333740234
sort_time 9.059906005859375e-06
build_wedge 4.7206878662109375e-05
collect_targets 0.014064788818359375
sort_time 7.62939453125e-06
build_wedge 2.6941299438476562e-05
collect_targets 0.013889074325561523
sort_time 6.4373016357421875e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.013896942138671875
sort_time 8.821487426757812e-06
build_wedge 2.6226043701171875e-05
collect_targets 0.013805627822875977
sort_time 9.298324584960938e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.01382589340209961
sort_time 8.821487426757812e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.013890981674194336
sort_time 8.344650268554688e-06
build_wedge 2.6226043701171875e-05
collect_targets 0.013960123062133789
sort_time 7.62939453125e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.013872385025024414
sort_time 7.62939453125e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.01385354995727539
sort_time 6.198883056640625e-06
build_we

collect_targets 0.015526771545410156
sort_time 7.867813110351562e-06
build_wedge 3.170967102050781e-05
collect_targets 0.015529870986938477
sort_time 6.67572021484375e-06
build_wedge 2.5987625122070312e-05
collect_targets 0.015086650848388672
sort_time 6.4373016357421875e-06
build_wedge 2.5510787963867188e-05
collect_targets 0.015255212783813477
sort_time 1.0728836059570312e-05
build_wedge 2.6702880859375e-05
collect_targets 0.015229463577270508
sort_time 7.62939453125e-06
build_wedge 2.6702880859375e-05
collect_targets 0.015062332153320312
sort_time 7.867813110351562e-06
build_wedge 2.6226043701171875e-05
collect_targets 0.015023231506347656
sort_time 8.106231689453125e-06
build_wedge 2.6941299438476562e-05
collect_targets 0.01499319076538086
sort_time 5.9604644775390625e-06
build_wedge 2.47955322265625e-05
collect_targets 0.014996528625488281
sort_time 5.0067901611328125e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.015053033828735352
sort_time 5.245208740234375e-06
build_

collect_targets 0.014673948287963867
sort_time 5.9604644775390625e-06
build_wedge 3.147125244140625e-05
collect_targets 0.013965845108032227
sort_time 4.5299530029296875e-06
build_wedge 2.6464462280273438e-05
collect_targets 0.013815641403198242
sort_time 4.5299530029296875e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.013941287994384766
sort_time 4.5299530029296875e-06
build_wedge 2.5033950805664062e-05
collect_targets 0.01388692855834961
sort_time 4.76837158203125e-07
build_wedge 2.5272369384765625e-05
collect_targets 0.01387166976928711
sort_time 4.76837158203125e-07
build_wedge 2.4557113647460938e-05
collect_targets 0.013805627822875977
sort_time 4.76837158203125e-07
build_wedge 2.47955322265625e-05
collect_targets 0.014010906219482422
sort_time 6.4373016357421875e-06
build_wedge 2.574920654296875e-05
collect_targets 0.013866662979125977
sort_time 6.4373016357421875e-06
build_wedge 2.574920654296875e-05
collect_targets 0.0138702392578125
sort_time 7.867813110351562e-06
b

collect_targets 0.01579880714416504
sort_time 4.76837158203125e-07
build_wedge 0.00014734268188476562
collect_targets 0.015106678009033203
sort_time 4.76837158203125e-07
build_wedge 2.4080276489257812e-05
collect_targets 0.014974832534790039
sort_time 2.384185791015625e-07
build_wedge 2.384185791015625e-05
collect_targets 0.015083074569702148
sort_time 4.76837158203125e-07
build_wedge 2.6464462280273438e-05
collect_targets 0.01513981819152832
sort_time 4.76837158203125e-07
build_wedge 2.4080276489257812e-05
collect_targets 0.014951229095458984
sort_time 4.76837158203125e-07
build_wedge 2.3603439331054688e-05
collect_targets 0.015248537063598633
sort_time 6.67572021484375e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.015004158020019531
sort_time 4.76837158203125e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.015246868133544922
sort_time 7.3909759521484375e-06
build_wedge 2.5272369384765625e-05
collect_targets 0.014913797378540039
sort_time 6.198883056640625e-06
buil

KeyboardInterrupt: 

In [ ]:
from tutorials.utils.tutorial_utils import visualize_history
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5006)